# Türk Hukuk RAG Chatbot

Bu notebook, `Renicames/turkish-law-chatbot` veri seti ile RAG boru hattının uçtan uca nasıl kurulduğunu örnekler.


In [ ]:
!pip -q install -r ../requirements.txt


### Alternatif: Türk Hukuk veri setini doğrudan yükle ve pandas'a çevir


In [ ]:
from datasets import load_dataset
import pandas as pd
import re

# Türk Hukuk datasetini HF Datasets ile yükle
# Not: Bu veri setinde Soru ve Cevap kolonları bulunur
hukuk = load_dataset("Renicames/turkish-law-chatbot", split="train")
print(hukuk)
print(hukuk.column_names)
print(hukuk[0])

# Pandas DataFrame'e çevir
df = hukuk.to_pandas()
print(df.head())

# Basit bir temizlik örneği (isteğe bağlı)
def basic_clean(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = re.sub(r"\s+", " ", t).strip()
    return t

df["Soru_clean"] = df["Soru"].map(basic_clean)
df["Cevap_clean"] = df["Cevap"].map(basic_clean)
df.head()


### (Opsiyonel) Haystack Document formatına dönüştürme


In [ ]:
# Eğer Haystack kullanacaksanız şu örnek işinize yarar
# pip install farm-haystack
try:
    from haystack.dataclasses import Document
    docs = [
        Document(content=row["Cevap_clean"], meta={"soru": row["Soru_clean"]})
        for _, row in df.iterrows()
        if isinstance(row["Cevap_clean"], str) and len(row["Cevap_clean"]) > 0
    ]
    print(docs[0] if docs else "no docs")
except Exception as e:
    print("Haystack kurulu değil veya Document importu başarısız:", e)


## 1) Veri Setini Yükle


In [ ]:
from datasets import load_dataset

ds = load_dataset('Renicames/turkish-law-chatbot')
ds


## 2) FAISS İndeksi Oluştur (script)


In [ ]:
%run ../src/build_index.py


## 3) Soru Sor ve Yanıt Al


In [ ]:
from src.rag_pipeline import answer_question

resp = answer_question('Anayasa madde 1 neyi düzenler?', top_k=5)
resp['answer']
